In [49]:
### Library Import Initialization
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm
import warnings
warnings.filterwarnings("ignore")


In [50]:
### Load in Stock Data
def import_stock_data(tickers, start_date):
    data = pd.DataFrame()
    if len([tickers]) == 1:
        data[tickers] = yf.download(tickers, start_date)['Adj Close']
        data = pd.DataFrame(data)
    else:
        for t in tickers:
            data[t] = yf.download(tickers, start_date)['Adj Close']
    return data

tickers = 'GOOG'
start_data = '2022-01-01'
stock_data = import_stock_data(tickers, start_data)
# Get most recent stock price
S_0 = stock_data.iloc[1, -1]
S_0

# Set the Strike Price (K) based on the current asset price (S_0)
K = 150

[*********************100%%**********************]  1 of 1 completed


In [51]:
### Compute Log Returns
def get_log_returns(data):
    return np.log(stock_data / stock_data.shift(1))

log_returns = get_log_returns(stock_data)
log_returns.tail()

,GOOG
Date,
2024-05-06,0.004958
2024-05-07,0.018378
2024-05-08,-0.010577
2024-05-09,0.002451
2024-05-10,-0.007547


In [52]:
### Calculate Historical Volatility
def volatility(log_returns):
    # Calculate standard deviation of log_returns
    std_dev = log_returns.std()
    # Calculate historical volatility (v_0) = sqrt(252) * std dev
    v_0 = np.sqrt(252) * std_dev # 252 trading days per year

    return v_0

v_0 = volatility(log_returns)
v_0

GOOG    0.34456
dtype: float64

In [53]:
### Initialize Parameters
# Mean-reversion speed in the Heston model
k = 0.1  
# Volatility of volatility in the Heston model
sigma = 0.2  
# Time step, typically representing trading days in a year
dt = 252  
# Long-term average variance (squared volatility) in the Heston model
theta = v_0 ** 2  
# Drift rate, computed as mean - 0.5 * variance
mean = log_returns.mean()  
var = log_returns.var()  
drift = mean - (0.5 * var)  


In [57]:
### Heston Model Function
def heston_model(S_0, K, v_0, drift, k, theta, sigma, dt, time_steps, trials):
    # Generate random numbers z_1 and z_2 from a standard normal distribution for each time step and trial
    z_1 = np.random.randn(time_steps, trials)
    z_2 = np.random.randn(time_steps, trials)
    
    # Create empty arrays to store asset prices and volatilities for each time step and trial
    S_t = np.zeros((time_steps, trials))
    v_t = np.zeros((time_steps, trials))

    # Set initial values for asset prices and volatilities
    S_t[0, :] = S_0
    v_t[0, :] = v_0

    # Loop through each time step starting from t=1
    for i in range(1, time_steps):
        # Calculate S_t for each time step and trial
        S_t[i, :] = S_t[i-1, :] + drift * S_t[i-1, :] * dt + np.sqrt(v_t[i-1, :] * dt) * S_t[i-1, :] * z_1[i, :]
        # Calculate v_t for each time step and trial
        v_t[i, :] = np.maximum(v_t[i-1, :] + k * (theta - v_t[i-1, :]) * dt + sigma * np.sqrt(v_t[i-1, :] * dt) * z_2[i, :], 0)

    # Calculate option prices using the simulated asset prices
    option_prices = np.maximum(S_t[-1, :] - K, 0)
    
    # Calculate the mean of the option prices (Monte Carlo estimate)
    option_price_mean = np.mean(option_prices)

    return option_price_mean

### Function Call
option_price_mean = heston_model(S_0, K, v_0, drift, k, theta, sigma, dt, time_steps = 252, trials = 100)
print("Option Price Estimate: ", option_price_mean)


ValueError: Length of values (100) does not match length of index (1)